In [6]:
import numpy as np
import pandas as pd
import os

In [7]:
source_path_wd = "/media/andres/T7 Shield1/U-CAN-Lymfom_A/Selected_for_UCAN_project"
source_path_bd = "/media/andres/T7 Shield/ucan_lymfom"

In [11]:
master_data_wd = pd.read_csv(os.path.join(source_path_wd, 'FinalSelected_exams_from_U-CAN-Lymfom_A_16Oct23.csv'))
master_data_wd[['system', 'npr', 'scan_date']]= master_data_wd['patient_directory'].str.split(pat='_|-', n=2, expand=True)
print(master_data_wd.shape)
master_data_wd.head(2)


(1316, 7)


,directory,source_directory,patient_directory,PET-CT_info,system,npr,scan_date
0,E:/U-CAN-Lymfom_A/Raw_DCM_transf_date_20221215...,E:/U-CAN-Lymfom_A/Raw_DCM_transf_date_20221215...,ASPTCTX0001_npr748236341307-20220908,PT-20220908-103801-3.11-WB-F18-FDG-VENFAS--THO...,ASPTCTX0001,npr748236341307,20220908
1,E:/U-CAN-Lymfom_A/Raw_DCM_transf_date_20221215...,E:/U-CAN-Lymfom_A/Raw_DCM_transf_date_20221215...,ASPTCTX0001_npr748236341307-20220908,CT-20220908-105049-3.11-WB-F18-FDG-VENFAS--THO...,ASPTCTX0001,npr748236341307,20220908


In [16]:
patient_list_wd = [item[:36] for item in master_data_wd.patient_directory.to_list()]
patient_list_wd[:2]

['ASPTCTX0001_npr748236341307-20220908',
 'ASPTCTX0001_npr748236341307-20220908']

In [9]:
master_data_bd = pd.read_excel(os.path.join(source_path_bd, "Finalized_dataset.xlsx"))
print(master_data_bd.shape)
master_data_bd.head(2)

(2264, 44)


,Unnamed: 0,Unnamed: 0_x,directory,source_directory,patient_directory,PET-CT_info,system,npr,scan_date,Unnamed: 0_y,...,modality,manufacturer,manufacturer_model,radiopharmaceutical,radiopharmaceutical_volume,radiopharmaceutical_start_time,radionuclide_total_dose,radionuclide_half_life,radionuclide_positron_fraction,radiopharmaceutical_start_date_time
0,0,2096,/media/andres/T7 Shield/ucan_lymfom/ASPTCTX000...,/media/andres/T7 Shield/ucan_lymfom,ASPTCTX0001_lpr385705046400-20140313,CT-20140313-152551-6.1_UAS-WB-FDG-3D-STANDARD_...,ASPTCTX0001,lpr385705046400,20140313,13743,...,CT,'GE MEDICAL SYSTEMS','Discovery STE',NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2095,/media/andres/T7 Shield/ucan_lymfom/ASPTCTX000...,/media/andres/T7 Shield/ucan_lymfom,ASPTCTX0001_lpr385705046400-20140313,PT-20140313-152728-UAS-WB-FDG-3D-UAS-WB-FDG-3D...,ASPTCTX0001,lpr385705046400,20140313,13745,...,PT,'GE MEDICAL SYSTEMS','Discovery STE','FDG -- fluorodeoxyglucose','0.0','141400.00','259344912.0','6588.0','0.97000002861023','20140313141400.00'


In [13]:
patient_list_bd = master_data_bd.patient_directory.to_list()
patient_list_bd[:2]

['ASPTCTX0001_lpr385705046400-20140313',
 'ASPTCTX0001_lpr385705046400-20140313']

In [22]:
patient_set_wd = set(patient_list_wd)
patient_set_bd = set(patient_list_bd)
extra_patient_in_wd = patient_set_wd.difference(patient_set_bd)
print(len(extra_patient_in_wd))
extra_patient_in_wd_final = [item2 for item1 in list(extra_patient_in_wd) for item2 in master_data_wd.patient_directory.to_list() if item1 in item2]
extra_patient_in_wd

201


{'ASPTCTX0001_npr116163692968-20201229',
 'ASPTCTX0001_npr134631340866-20180413',
 'ASPTCTX0001_npr134631340866-20200204',
 'ASPTCTX0001_npr138028683671-20200211',
 'ASPTCTX0001_npr143944474227-20210119',
 'ASPTCTX0001_npr159135068459-20200604',
 'ASPTCTX0001_npr183027863262-20180507',
 'ASPTCTX0001_npr183406378492-20220223',
 'ASPTCTX0001_npr184555245175-20160919',
 'ASPTCTX0001_npr184555245175-20170307',
 'ASPTCTX0001_npr184555245175-20191219',
 'ASPTCTX0001_npr190483255078-20210203',
 'ASPTCTX0001_npr196454688576-20141201',
 'ASPTCTX0001_npr212313157683-20180518',
 'ASPTCTX0001_npr216632833659-20200703',
 'ASPTCTX0001_npr218702301637-20170320',
 'ASPTCTX0001_npr218702301637-20170607',
 'ASPTCTX0001_npr218702301637-20220126',
 'ASPTCTX0001_npr235266377960-20140807',
 'ASPTCTX0001_npr252097509348-20170224',
 'ASPTCTX0001_npr252097509348-20190111',
 'ASPTCTX0001_npr252097509348-20200804',
 'ASPTCTX0001_npr252097509348-20210209',
 'ASPTCTX0001_npr252097509348-20210518',
 'ASPTCTX0001_np

In [25]:
for item in extra_patient_in_wd_final:
    if 'JK' in item:
        print(item)

ASPTCTX0001_npr134631340866-20200204____JK
ASPTCTX0001_npr134631340866-20200204____JK
ASPTCTX0001_npr897251392030-20190613____JK_HIT
ASPTCTX0001_npr897251392030-20190613____JK_HIT


In [27]:
extra_patient_wd_df = master_data_wd[master_data_wd['patient_directory'].isin(extra_patient_in_wd_final)]
print(extra_patient_wd_df.shape)
extra_patient_wd_df.head(2)

(402, 7)


,directory,source_directory,patient_directory,PET-CT_info,system,npr,scan_date
10,E:/U-CAN-Lymfom_A/Raw_DCM_transf_date_20221205...,E:/U-CAN-Lymfom_A/Raw_DCM_transf_date_20221205...,ASPTCTX0001_npr826756003305-20220810,CT-20220810-114419-3.12-WB-F18-FDG-VENFAS--THO...,ASPTCTX0001,npr826756003305,20220810
11,E:/U-CAN-Lymfom_A/Raw_DCM_transf_date_20221205...,E:/U-CAN-Lymfom_A/Raw_DCM_transf_date_20221205...,ASPTCTX0001_npr826756003305-20220810,PT-20220810-113359-3.12-WB-F18-FDG-VENFAS--THO...,ASPTCTX0001,npr826756003305,20220810


In [29]:
extra_patient_wd_npr = extra_patient_wd_df.npr.to_list()
extra_patient_wd_npr[:2]

['npr826756003305', 'npr826756003305']

In [32]:
extra_patient_bd_npr_df = master_data_bd[master_data_bd['npr'].isin(extra_patient_wd_npr)]
print(extra_patient_bd_npr_df.shape)
extra_patient_bd_npr_df.head(2)

(154, 44)


,Unnamed: 0,Unnamed: 0_x,directory,source_directory,patient_directory,PET-CT_info,system,npr,scan_date,Unnamed: 0_y,...,modality,manufacturer,manufacturer_model,radiopharmaceutical,radiopharmaceutical_volume,radiopharmaceutical_start_time,radionuclide_total_dose,radionuclide_half_life,radionuclide_positron_fraction,radiopharmaceutical_start_date_time
276,276,1991,/media/andres/T7 Shield/ucan_lymfom/ASPTCTX000...,/media/andres/T7 Shield/ucan_lymfom,ASPTCTX0001_npr196454688576-20150127,CT-20150127-132952-6.1_UAS-WB-FDG-3D-STANDARD_...,ASPTCTX0001,npr196454688576,20150127,8127,...,CT,'GE MEDICAL SYSTEMS','Discovery STE',NaN,NaN,NaN,NaN,NaN,NaN,NaN
277,277,1992,/media/andres/T7 Shield/ucan_lymfom/ASPTCTX000...,/media/andres/T7 Shield/ucan_lymfom,ASPTCTX0001_npr196454688576-20150127,PT-20150127-133127-UAS-WB-FDG-3D-UAS-WB-FDG-3D...,ASPTCTX0001,npr196454688576,20150127,8129,...,PT,'GE MEDICAL SYSTEMS','Discovery STE','FDG -- fluorodeoxyglucose','0.0','121700.00','310023968.0','6588.0','0.97000002861023','20150127121700.00'
